- date: 2022-05-08 11:17:17
- author: Jerry Su
- slug: Paddle-gather-index_select
- title: Paddle gather index_select
- category: 
- tags: Paddle

gather实现torch数组花式索引

https://github.com/PaddlePaddle/Paddle/issues/42554 [受到启发]如果要多个list做索引建议一个一个来分开处理

https://github.com/PaddlePaddle/Paddle/issues/35072

https://github.com/PaddlePaddle/Paddle/pull/34824

https://github.com/PaddlePaddle/Paddle/issues/33605

In [141]:
import torch

In [142]:
x = torch.randn(2, 3, 4, 5)
print(x.shape)
x

torch.Size([2, 3, 4, 5])


tensor([[[[-5.4468e-01, -5.0562e-01,  1.2807e+00,  3.4061e-01,  2.0514e+00],
          [-1.3104e+00, -9.8287e-01,  3.7319e-01, -1.4745e+00,  1.4230e+00],
          [-4.3422e-01,  2.3498e+00,  1.3136e-01,  2.1823e+00,  5.1223e-01],
          [ 1.2024e+00, -9.4088e-01,  1.2282e+00, -1.0885e+00, -8.7535e-02]],

         [[ 7.8700e-01, -2.2048e-01, -6.0638e-01,  9.3186e-01,  5.6504e-02],
          [ 1.7838e+00, -8.2962e-01, -2.7352e+00, -1.4160e+00,  2.6104e+00],
          [-3.7266e-02, -2.1329e-01, -2.1592e+00,  2.2810e+00, -2.3482e+00],
          [-2.9233e-01,  1.1370e+00,  1.3566e+00,  1.9546e-01, -1.8684e-01]],

         [[-1.8817e-01,  4.0500e-01, -1.4191e+00,  2.3311e-01,  1.3099e+00],
          [-1.3656e+00,  1.0606e+00,  1.7107e+00, -1.9834e-01,  4.1514e-01],
          [ 2.2179e-01, -7.2007e-01, -9.1608e-01,  1.4468e+00,  4.3706e-02],
          [ 6.4846e-01, -1.3944e+00,  4.1428e-01,  4.0398e-01,  5.4115e-01]]],


        [[[-2.3598e+00, -8.8833e-01, -4.2781e-01, -2.3981e-01,  2.14

In [143]:
dim0 = torch.tensor([0, 1])
dim2 = torch.tensor([0, 2])

In [144]:
x_ = x[dim0, :, dim2]
print(x_.shape)
x_

torch.Size([2, 3, 5])


tensor([[[-0.5447, -0.5056,  1.2807,  0.3406,  2.0514],
         [ 0.7870, -0.2205, -0.6064,  0.9319,  0.0565],
         [-0.1882,  0.4050, -1.4191,  0.2331,  1.3099]],

        [[ 0.0924,  0.7283,  2.2149,  1.1706, -0.0477],
         [-1.2955,  0.5662, -0.1063, -0.0722,  1.1317],
         [-1.0909, -0.9825, -0.9295,  1.4528, -0.3839]]])

## paddle实现

In [145]:
import paddle

In [146]:
xx = paddle.to_tensor(x.numpy())
# xx

In [147]:
axis0 = paddle.to_tensor([0, 1])
axis1 = paddle.to_tensor([0, 2])

/opt/conda/envs/blog/lib/python3.8/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [121]:
out1 = paddle.gather(xx, index=axis0, axis=0)

In [139]:
index =paddle.stack([axis0, axis1], axis=-1)
index 

Tensor(shape=[2, 2], dtype=int64, place=CPUPlace, stop_gradient=True,
       [[0, 0],
        [1, 2]])

In [140]:
paddle.gather_nd(out1, index=index)#.transpose([0, 1, 3, 2])

Tensor(shape=[2, 4, 5], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[-0.07601893, -0.95786285, -1.43674135, -1.93300319,  1.93319392],
         [-0.51922202, -1.28075957, -0.90560406, -1.81705618,  2.47172594],
         [-0.35581169,  0.16289686, -2.86410379,  1.34724343, -0.92106235],
         [ 0.09319570,  0.64727783,  1.04392993,  1.27829981,  0.10877635]],

        [[-0.76419204, -1.69635403,  1.48616660,  0.87069529, -0.39233911],
         [ 1.40529430, -0.20886442,  1.51489639,  1.34371901, -0.73188514],
         [ 0.56418574,  0.59838581,  0.32179439, -1.31146288,  0.31410775],
         [-1.38033020,  0.01542827,  0.43722150,  0.47403052, -1.02283168]]])